# Two Electron Products

1. Create products of the benzene occupied states on the real-space grid and output to cube file

2. Perform the expansions using interpolation points and functions:
* Visually compare
* Numerically compare: 
    * Avg, Min and Max Diff of the functions
    * Integral of the functions over the volume

## Definitions 

|----------|-------------------------------------------------------------------------------------|
| Quantity | Definition                                                                          |
|----------|-------------------------------------------------------------------------------------|
| $N_e$    | Number of electrons. Should be equivalent to the number of occupied basis functions |
| $N_g$    | Number of real-space grid points                                                    |
| $N_\mu$  | Number of interpolation points/vectors                                              |

|----------|-------------------------------------------------------------------------------------|
| Matrix   | Dimensions                                                                          |
|----------|-------------------------------------------------------------------------------------|
| $\phi(\mathbf{r})$      |  Kohn-Sham orbitals                                                  |
| $\psi(\mathbf{r})$      |  Kohn-Sham orbitals obtained if one does an outer SCF loop.i.e. self-consistent w.r.t. $V_{EX}$  |
| $\zeta_\mu(\mathbf{r})$ |  Auxiliary basis functions                                           |
| $Z$                     |  Product state matrix of shape $(N_g, N_e^2)$, where an element (k, ij) corresponds to $\phi_i(\mathbf{r}_k) \psi_j(\mathbf{r}_k)$ |
| $C$                     |  Expansion coefficient matrix of shape $(N_g, N_e^2)$                |
| $\Theta$                |  Matrix of auxiliary basis functions, with shape $(N_g, N_\mu)$      |



## Decomposing the Contractions $(ZC^T)$ and $(CC^T)^{-1}$

These notes follow the paper:
> Interpolative Separable Density Fitting Decomposition for Accelerating Hybrid Density Functional Calculations with Applications to Defects in Silicon
[J. Chem. Theory Comput. 2017, 13, 5420-5431](https://pubs.acs.org/doi/10.1021/acs.jctc.7b00807)

### Optimal Construction of $(ZC^T)$

The contraction $(ZC^T)$ has shape $(N_g, N_e^2)(N_e^2, N_\mu) \rightarrow (N_g, N_\mu)$. This can be defined in terms of quasi density-matrices:

\begin{align}
  (ZC^T)_{k,\mu} = P^\Phi (\mathbf{r}_k, \mathbf{r}_\mu) P^\Psi (\mathbf{r}_k, \mathbf{r}_\mu),
\end{align}

where $k$ is an index over the real-space grid and $\mu$ is an index over the interpolation points. The whole contraction is defined in terms of the Hadamard (element-wise) product between the two quasi density matrices:

\begin{align}
  (ZC^T)= P^\Phi \odot P^\Psi.
\end{align}

$(ZC^T)$, $P^\Phi (\mathbf{r}_k, \mathbf{r}_\mu)$ and $P^\Psi (\mathbf{r}_k, \mathbf{r}_\mu)$ cleatly all have the same shape, $(N_g, N_\mu)$. The quasi density-matrices are themselves given as:

\begin{align}
  P^\Phi(\mathbf{r}_k, \mathbf{r}_\mu) = \sum^m_{i=1} \phi(\mathbf{r}_k) \phi(\mathbf{r}_\mu) \\
  P^\Psi(\mathbf{r}_k, \mathbf{r}_\mu) = \sum^n_{i=1} \psi(\mathbf{r}_k) \psi(\mathbf{r}_\mu)
\end{align}

where $m$ typically equals $N_e$ (the number of occupied states), and $n \ge N_e$ depending on whether $\{\phi\} = \{\psi\}$. The two sets of KS orbitals will only differ if an outer SCF loop is used to generate $\{\psi\}$, in which case they may contain some unoccupied states.

If one writes the defintion of $P^\Phi(\mathbf{r}_k, \mathbf{r}_\mu)$ more compactly:

\begin{align}
  P^\Phi_{k,\mu} = \sum^m_{i=1} \phi_{k, i} \phi_{\mu i}
\end{align}

one can see that:

\begin{align}
  P^\Phi = \Phi (\Phi')^T
\end{align}

where $\Phi'$ is the KS orbital matrix, with the rows restricted to interpolation points. In the case that $\{\phi\} = \{\psi\}$, this simplifies to:

\begin{align}
  (ZC^T)&= \left[P^\Phi\right]^2 \\
        &= \left[\Phi (\Phi')^T \right]^2
\end{align}

where the $^2$ implies element-wise squaring.

**TODO**: Add the number of flops per operation


### Optimal Construction of $(CC^T)^{-1}$

The coefficient matrix $C$ has shape $(N_\mu, N_e^2)$, so $(CC^T)$ will have shape $(N_\mu, N_\mu$, as will its inverse. As such, the decomposable product is defined as:

\begin{align}
  (CC^T)_{\nu,\mu} = P^\Phi (\mathbf{r}_\nu, \mathbf{r}_\mu) P^\Psi (\mathbf{r}_\nu, \mathbf{r}_\mu),
\end{align}

where both $\nu$ and $\mu$ run over the interpolation points only. 

### Optimal Construction of $\Theta$

To compute $\Theta = (ZC^T)(CC^T)^{-1}$, I *assume* one therefore needs to compute the contraction $(CC^T)$, compute its inverse (perhaps via SVD),
then perform the final matrix multiplication (contaction over $N_\mu$):

\begin{align}
  \Theta = \left[ (ZC^T) \right] \left[(CC^T)^{-1}\right].
\end{align}

$\Theta$ will have shape $(N_g, N_\mu) (N_\mu, N_\mu) \rightarrow (N_g, N_\mu)$, such that each column corresponds to an interpolation vector.

### Construction of Approximate Pair Product States

Finally, one can test:

\begin{align}
  \phi_i(\mathbf{r}) \psi_j(\mathbf{r}) \approx \sum_\mu^{N_\mu} \zeta_\mu(\mathbf{r})  \phi_i(\mathbf{r}_\mu) \psi_j(\mathbf{r}_\mu) 
\end{align}

To construct the product state matrix $A$, one uses the face-splitting product on arrays of $\phi$ and $\psi$. This is implemented in [isdf_vectors.py](../src/isdf_prototypes/isdf_vectors.py). One then performs the matrix-matrix product $\zeta A$, where $A$ has shape $(N_\mu, N_e^2)$, $\zeta$ has shape $(N_g, N_\mu)$, and the final contraction has shape $(N_g, N_e^2)$
